In [22]:

import time
from collections import defaultdict, deque
from typing import Tuple, Dict, List, Deque

class OrderBook:
    def __init__(self, tick_size: float, lot_size: int):
        # OBJECTIF : Initialise un carnet d'ordres avec une taille de tick et une taille de lot spécifiées.

        self.tick_size = tick_size
        self.lot_size = lot_size
        self.buy_orders = defaultdict(deque)
        self.sell_orders = defaultdict(deque)
        self.transaction_history = []

    def add_order(self, order_type: str, price: float, quantity: int, participant_id: str, role: str):
       # OBJECTIF : Ajoute un ordre d'achat ou de vente au carnet d'ordres.

        rounded_price = round(price / self.tick_size) * self.tick_size

        if quantity % self.lot_size != 0:
            raise ValueError(f"Erreur: La quantité {quantity} n'est pas un multiple du lot {self.lot_size}.")

        # éxécution des ordres type taker
        if role == 'taker':
            remaining_quantity = quantity
            order_timestamp = time.time()

            if order_type == 'buy':
                sorted_sell_orders = sorted(self.sell_orders.items(), key=lambda x: x[0])

                # boucle qui éxécute l'ordre tant que la quantité demandée n'est pas atteinte
                while remaining_quantity > 0 and sorted_sell_orders:
                    for sell_price, sell_orders in sorted_sell_orders:
                        while sell_orders and remaining_quantity > 0:
                            sell_quantity, seller_id, order_timestamp_sell = sell_orders.popleft()

                            if remaining_quantity >= sell_quantity:
                                remaining_quantity -= sell_quantity
                                self.transaction_history.append((participant_id, 'buy', sell_price, sell_quantity))
                            else:
                                new_quantity = sell_quantity - remaining_quantity
                                sell_orders.appendleft((new_quantity, seller_id, order_timestamp_sell))
                                self.transaction_history.append((participant_id, 'buy', sell_price, remaining_quantity))
                                remaining_quantity = 0
                                break

                    if remaining_quantity == 0:
                        break

            elif order_type == 'sell':
                sorted_buy_orders = sorted(self.buy_orders.items(), key=lambda x: x[0], reverse=True)

                # boucle qui éxécute l'ordre tant que la quantité proposée n'est pas atteinte
                while remaining_quantity > 0 and sorted_buy_orders:
                    for buy_price, buy_orders in sorted_buy_orders:
                        while buy_orders and remaining_quantity > 0:
                            buy_quantity, buyer_id, order_timestamp_buy = buy_orders.popleft()

                            if remaining_quantity >= buy_quantity:
                                remaining_quantity -= buy_quantity
                                self.transaction_history.append((participant_id, 'sell', buy_price, buy_quantity))
                            else:
                                new_quantity = buy_quantity - remaining_quantity
                                buy_orders.appendleft((new_quantity, buyer_id, order_timestamp_buy))
                                self.transaction_history.append((participant_id, 'sell', buy_price, remaining_quantity))
                                remaining_quantity = 0
                                break

                    if remaining_quantity == 0:
                        break
        
        # ajout au carnet des ordres type marker
        else:
            order = (quantity, participant_id, time.time())
            if order_type == 'buy':
                self.buy_orders[rounded_price].append(order)
            elif order_type == 'sell':
                self.sell_orders[rounded_price].append(order)

    def cancel_order(self, order_type: str, price: float, quantity: int, participant_id: str):
       # OBJECTIF : Annulation d'un ordre d'achat ou de vente du carnet d'ordres.

        rounded_price = round(price / self.tick_size) * self.tick_size

        if quantity % self.lot_size != 0:
            raise ValueError(f"Erreur: La quantité {quantity} n'est pas un multiple du lot {self.lot_size}.")

        # suppression dans le carnet
        if order_type == 'buy':
            if rounded_price in self.buy_orders:
                order_list = self.buy_orders[rounded_price]
                for order in order_list.copy():
                    if order[0] == quantity and order[1] == participant_id:
                        order_list.remove(order)
                        if not order_list:
                            del self.buy_orders[rounded_price]
                        break
        elif order_type == 'sell':
            if rounded_price in self.sell_orders:
                order_list = self.sell_orders[rounded_price]
                for order in order_list.copy():
                    if order[0] == quantity and order[1] == participant_id:
                        order_list.remove(order)
                        if not order_list:
                            del self.sell_orders[rounded_price]
                        break

    def fixing(self) -> float:
        # OBJECTIF : Détermine le prix de fixing et exécute les ordres correspondants.

        # Trier les ordres d'achat et de vente selon leur prix
        sorted_buy_orders = sorted(((price, orders) for price, orders in self.buy_orders.items()), reverse=True)
        sorted_sell_orders = sorted(((price, orders) for price, orders in self.sell_orders.items()))

        # Calculer les quantités cumulées pour chaque prix
        buy_cumulative_quantities = []
        sell_cumulative_quantities = []

        buy_quantity = 0
        for price, orders in sorted_buy_orders:
            buy_quantity += sum(order[0] for order in orders)
            buy_cumulative_quantities.append((price, buy_quantity))

        sell_quantity = 0
        for price, orders in sorted_sell_orders:
            sell_quantity += sum(order[0] for order in orders)
            sell_cumulative_quantities.append((price, sell_quantity))

        # Determination du prix pour lequel il y a le plus gros volume d'echange
        max_traded_quantity = 0
        fixing_price = None

        buy_index = len(buy_cumulative_quantities) - 1
        sell_index = 0

        while buy_index >= 0 and sell_index < len(sell_cumulative_quantities):
            buy_price, buy_quantity = buy_cumulative_quantities[buy_index]
            sell_price, sell_quantity = sell_cumulative_quantities[sell_index]

            if buy_price == sell_price:
                traded_quantity = min(buy_quantity, sell_quantity)
                fixing_price = buy_price

                if traded_quantity > max_traded_quantity:
                    max_traded_quantity = traded_quantity
                    fixing_price = buy_price

                buy_index -= 1
                sell_index += 1

            elif buy_price < sell_price:
                buy_index -= 1
            else:
                sell_index += 1

            # sortie de la boucle une fois le prix trouvé
            if fixing_price is not None:
                break

        # Exécuter les ordres qui matchent lors du fixing
        orders_to_execute = []
    
        # Ajouter les ordres de vente inférieurs ou égaux au prix de fixing
        for sell_price, sell_orders in sorted_sell_orders:
           if sell_price <= fixing_price:
                orders_to_execute.extend([(sell_price, 'sell', order) for order in sell_orders])
           else:
               break

         # Ajouter les ordres d'achat supérieurs ou égaux au prix de fixing
        for buy_price, buy_orders in sorted_buy_orders:
            if buy_price >= fixing_price:
               orders_to_execute.extend([(buy_price, 'buy', order) for order in buy_orders])
            else:
               break
            
        # Exécuter les ordres
        remaining_sell_quantity = max_traded_quantity
        remaining_buy_quantity = max_traded_quantity
        executed_orders = []
            
        for price, order_type, order in sorted(orders_to_execute, key=lambda x: (x[0], x[2][2])):
            quantity, participant_id, order_timestamp = order
            if order_type == 'sell':
                if remaining_sell_quantity >= quantity:
                    executed_orders.append((price, order_type, order))
                    remaining_sell_quantity -= quantity
                else:
                    new_quantity = quantity - remaining_sell_quantity
                    executed_orders.append((price, order_type, (remaining_sell_quantity, participant_id, order_timestamp)))
                    self.cancel_order(order_type, price, quantity, participant_id)
                    self.sell_orders[price].append((new_quantity, participant_id, order_timestamp))
                    remaining_sell_quantity = 0
            elif order_type == 'buy':
                if remaining_buy_quantity >= quantity:
                    executed_orders.append((price, order_type, order))
                    remaining_buy_quantity -= quantity
                else:
                    new_quantity = quantity - remaining_buy_quantity
                    executed_orders.append((price, order_type, (remaining_buy_quantity, participant_id, order_timestamp)))
                    self.buy_orders[price].append((new_quantity, participant_id, order_timestamp))
                    remaining_buy_quantity = 0
            
        for price, order_type, order in executed_orders:
            quantity, participant_id, order_timestamp = order
            self.cancel_order(order_type, price, quantity, participant_id)
            self.transaction_history.append((participant_id, order_type, fixing_price, quantity))
                    
        return fixing_price

    
    def get_transaction_history(self) -> List[Tuple[str, str, float, int]]:
        # OBJECTIF : Récupèration de l'historique des transactions.

        return self.transaction_history


    def print_order_book(self):
        # OBJECTIF : Afficher le carnet d'ordres actuel.
     
        print("\nCarnet d'ordres actuel :")
        print("{:^20} | {:^10} | {:^20}".format("Quantité à acheter", "Prix", "Quantité à vendre"))
    
        all_orders = []
    
        # Récupérer tous les ordres, qu'ils soient d'achat ou de vente
        for price, orders in self.buy_orders.items():
            for order in orders:
                all_orders.append((order[0], price, 0))
    
        for price, orders in self.sell_orders.items():
            for order in orders:
                all_orders.append((0, price, order[0]))
    
        # Trier tous les ordres par prix croissant
        sorted_orders = sorted(all_orders, key=lambda x: x[1])
    
        # Afficher les ordres triés
        for order in sorted_orders:
            quantity_buy, price, quantity_sell = order
            print("{:^20} | {:^10.2f} | {:^20}".format(quantity_buy, price, quantity_sell))
    
                
# Exemple d'utilisation
order_book = OrderBook(tick_size=0.01, lot_size=10)

# Ajouter des ordres d'achat (makers)
order_book.add_order('buy', 100.05, 100, 'participant1', 'maker')
order_book.add_order('buy', 101.00, 50, 'participant2', 'maker')
order_book.add_order('buy', 99.95, 150, 'participant3', 'maker')
order_book.add_order('buy', 100.03, 50, 'participant9', 'maker')

# Ajouter des ordres de vente (makers)
order_book.add_order('sell', 100.03, 120, 'participant4', 'maker')
order_book.add_order('sell', 101.04, 80, 'participant5', 'maker')
order_book.add_order('sell', 100.05, 90, 'participant6', 'maker')
order_book.add_order('sell', 99.30, 90, 'participant7', 'maker')

# Afficher le carnet d'ordres actuel
order_book.print_order_book()

# Exécuter le fixing
fixing_price = order_book.fixing()
print(f"Prix de fixing : {fixing_price}")

# Ajouter un ordre d'achat (taker)
#order_book.add_order('buy', 100.20, 200, 'participant8', 'taker')

# Afficher l'historique des transactions
print("\nHistorique des transactions:")
for transaction in order_book.get_transaction_history():
    print(transaction)

# Afficher le carnet d'ordres après les transactions
order_book.print_order_book()


Carnet d'ordres actuel :
 Quantité à acheter  |    Prix    |  Quantité à vendre  
         0           |   99.30    |          90         
        150          |   99.95    |          0          
         50          |   100.03   |          0          
         0           |   100.03   |         120         
        100          |   100.05   |          0          
         0           |   100.05   |          90         
         50          |   101.00   |          0          
         0           |   101.04   |          80         
Prix de fixing : 100.03

Historique des transactions:
('participant7', 'sell', 100.03, 90)
('participant9', 'buy', 100.03, 50)
('participant4', 'sell', 100.03, 110)
('participant1', 'buy', 100.03, 100)
('participant2', 'buy', 100.03, 50)

Carnet d'ordres actuel :
 Quantité à acheter  |    Prix    |  Quantité à vendre  
        150          |   99.95    |          0          
         0           |   100.03   |          10         
         0           |   1